In [1]:
!pip3 install auto-sklearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 42.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.1 MB/s eta 0:00:00
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641936 sha256=41fe8d6f94eda46db89daacd6387ce2f6b01188e5950726dcdb7dd3ae8581483
  Stored in directory: /root/.cache/pip/wheels/2b/0b/d7/8f

In [2]:
# Tree-based Pipeline Optimization Tool
! pip install tpot

In [3]:
import sys  
print(sys.version)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [4]:
import platform  
print(platform.platform())  
print(platform.system())  
print(platform.release())

Linux-5.15.154+-x86_64-with-debian-bullseye-sid
Linux
5.15.154+


In [5]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed bhi
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 6

In [6]:
try :
    import autosklearn
except :
    try :
        import autosklearn
    except :
        pass

In [7]:
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.classification import AutoSklearnClassifier
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering
from sklearn.feature_selection import RFECV
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, accuracy_score, precision_score, recall_score, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.svm import SVC
from lightgbm import LGBMClassifier,  LGBMRegressor
from bayes_opt import BayesianOptimization

In [8]:
df_fire = pd.read_csv("/kaggle/input/forest-fire-risk-prediction/gds_fire_30mm.csv")
df_nofire = pd.read_csv("/kaggle/input/forest-fire-risk-prediction/gds_nofire_30mm.csv")

df = pd.concat([df_fire,df_nofire])

features = ['system:index', 'days', 'evisum', 'ndwisum', 'ndvisum', 'lstsum', '.geo', 'time']
df = df.drop(features, axis = 1)

df = df[df.apply(lambda row: row.isin([-99999.000000]).sum() == 0, axis=1)] 

# 加载数据集并进行标准化处理
X = df.drop(['labels'], axis = 1).values
y = df['labels'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
  
X_resampled, y_resampled = resample(X_scaled, y, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

NameError: name 'resample' is not defined

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
predictions = automl.predict(X_test_resampled)

In [ ]:
accuracy = accuracy_score(y_test_resampled, predictions).round(5)
precision = precision_score(y_test_resampled, predictions).round(5)
recall = recall_score(y_test_resampled, predictions).round(5) 
f1 = f1_score(y_test_resampled, predictions).round(5)
roc_auc = roc_auc_score(y_test_resampled, predictions).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif   
from xgboost import XGBClassifier
from sklearn.feature_selection import RFECV
import lightgbm as lgbm
from sklearn.base import clone  
import numpy as np  
  
# 定义基于互信息增益的初步特征选择器  
def select_features_by_mi(X, y, k):  
    selector = SelectKBest(mutual_info_classif, k=k)  
    X_new = selector.fit_transform(X, y)  
    selected_feature_indices = np.where(selector.get_support())[0]  
    return X_new, selected_feature_indices  
  
# 定义基于RFECV的特征选择器  
def select_features_by_rfecv(X, y, estimator, cv=5):  
    selector = RFECV(estimator=estimator, step=1, cv=cv, scoring='accuracy')  
    selector.fit(X, y)  
    selected_feature_indices = np.where(selector.support_)[0]  
    return selector.transform(X), selected_feature_indices  
  
# 结合互信息增益和RFECV的特征选择算法  
def combined_feature_selection(X, y, k_mi, estimator, cv=5, n_repeats=10):  
    selected_feature_indices_all_runs = []  
      
    # 重复运行特征选择以提高稳定性  
    for repetition in range(n_repeats):  
        # 第一阶段：使用互信息增益选择特征  
        X_mi, selected_feature_indices_mi = select_features_by_mi(X, y, k_mi)  
          
        # 第二阶段：在互信息增益选择的特征上使用RFECV进一步选择特征  
        if X_mi.shape[1] > 1:  
            X_rfecv, selected_feature_indices_rfecv = select_features_by_rfecv(X_mi, y, clone(estimator), cv=cv)  
            # 将RFECV选择的特征映射回原始特征索引  
            selected_feature_indices = selected_feature_indices_mi[selected_feature_indices_rfecv]  
        else:  
            selected_feature_indices = selected_feature_indices_mi  
            X_rfecv = X_mi  
          
        selected_feature_indices_all_runs.append(selected_feature_indices)  
      
    # 聚合多次运行的结果，例如通过投票  
    feature_votes = np.zeros(X.shape[1], dtype=int)  
    for feature_indices in selected_feature_indices_all_runs:  
        feature_votes[feature_indices] += 1  
      
    # 选择在大多数运行中都被选中的特征，例如超过一半的运行  
    selected_feature_indices_final = np.where(feature_votes > (n_repeats / 2))[0]  
      
    return X[:, selected_feature_indices_final], selected_feature_indices_final  
  
# 选择一个估计器 
estimator = lgbm.LGBMClassifier(random_state=42) 
  
# 执行特征选择  
X_selected, selected_feature_indices = combined_feature_selection(X_scaled, y, k_mi=16, estimator=estimator)  
print("Selected feature indices:", selected_feature_indices)

In [ ]:
feature_migd_rfecv = df.columns.drop(['labels'])[[selected_feature_indices]].values
feature_migd_rfecv

In [ ]:
migd_rfecv_df = pd.concat([df[feature_migd_rfecv], df['labels']], axis=1)

In [ ]:
# 加载数据集并进行标准化处理
X_migd_rfecv = migd_rfecv_df.drop(['labels'], axis = 1).values
y_migd_rfecv = migd_rfecv_df['labels'].values
scaler = StandardScaler()
X_scaled_migd_rfecv = scaler.fit_transform(X_migd_rfecv)

X_resampled, y_resampled = resample(X_scaled_migd_rfecv, y_migd_rfecv, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
pred = automl.predict(X_test_resampled)
accuracy = accuracy_score(y_test_resampled, pred).round(5)
precision = precision_score(y_test_resampled, pred).round(5)
recall = recall_score(y_test_resampled, pred).round(5) 
f1 = f1_score(y_test_resampled, pred).round(5)
roc_auc = roc_auc_score(y_test_resampled, pred).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')

In [ ]:
from sklearn.linear_model import Lasso 

lasso = Lasso(alpha=0.1)  # alpha是正则化强度的参数  
lasso.fit(X_scaled, y)  
  
# 获取特征系数  
coef = lasso.coef_  
  
# 找出被选择的特征（系数不为零的特征）  
selected_features_lasso = [i for i, coef_i in enumerate(coef) if coef_i != 0]  
  
print("Selected features:", selected_features_lasso)  

In [ ]:
feature_lasso = df.columns.drop(['labels'])[[selected_features_lasso]].values
feature_lasso

In [ ]:
lasso_df = pd.concat([df[feature_lasso], df['labels']], axis=1)

In [ ]:
# 加载数据集并进行标准化处理
X_lasso = lasso_df.drop(['labels'], axis = 1).values
y_lasso = lasso_df['labels'].values
scaler = StandardScaler()
X_scaled_lasso = scaler.fit_transform(X_lasso)

X_resampled, y_resampled = resample(X_scaled_lasso, y_lasso, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
pred_lasso = automl.predict(X_test_resampled)
accuracy = accuracy_score(y_test_resampled, pred_lasso).round(5)
precision = precision_score(y_test_resampled, pred_lasso).round(5)
recall = recall_score(y_test_resampled, pred_lasso).round(5) 
f1 = f1_score(y_test_resampled, pred_lasso).round(5)
roc_auc = roc_auc_score(y_test_resampled, pred_lasso).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')

In [ ]:
selector = SelectKBest(mutual_info_classif, k=16)  
  
# 训练选择器  
X_new = selector.fit_transform(X_scaled, y)  
  
# 获取被选择的特征索引  
selected_indices_mir = selector.get_support(indices=True)  
  
# 输出被选择的特征索引  
print("Selected feature indices:", selected_indices_mir)  

In [ ]:
feature_mir = df.columns.drop(['labels'])[[selected_indices_mir]].values
feature_mir

In [ ]:
mir_df = pd.concat([df[feature_mir], df['labels']], axis=1)

# 加载数据集并进行标准化处理
X_mir = mir_df.drop(['labels'], axis = 1).values
y_mir = mir_df['labels'].values
scaler = StandardScaler()
X_scaled_mir = scaler.fit_transform(X_mir)

X_resampled, y_resampled = resample(X_scaled_mir, y_mir, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
pred_mir = automl.predict(X_test_resampled)
accuracy = accuracy_score(y_test_resampled, pred_mir).round(5)
precision = precision_score(y_test_resampled, pred_mir).round(5)
recall = recall_score(y_test_resampled, pred_mir).round(5) 
f1 = f1_score(y_test_resampled, pred_mir).round(5)
roc_auc = roc_auc_score(y_test_resampled, pred_mir).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')

In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier

def feature_selection_mirdtr(X, y):
    # 计算每个特征与目标变量之间的互信息得分
    selector = SelectKBest(score_func=mutual_info_classif, k=18)
    X_new = selector.fit_transform(X, y)
    
    # 构建决策树回归模型
    tree = DecisionTreeClassifier(random_state = 42)
    tree.fit(X_new, y)
    
    # 输出特征重要性
    feature_importances = tree.feature_importances_
        
    # 根据重要性排序特征索引
    sorted_indices_mirdtr = feature_importances.argsort()[::-1]
    
    # 输出特征排序结果
    for i in sorted_indices_mirdtr:
        print(f"Feature {i}: Importance {feature_importances[i]}")
        
    # 返回排名前k个重要特征的索引
    R = 12  
    selected_indices_mirdtr = sorted_indices_mirdtr[:R]
    return selected_indices_mirdtr

In [ ]:
selected_indices_mirdtr = feature_selection_mirdtr(X_scaled, y)
feature_mirdtr = df.columns.drop(['labels'])[[selected_indices_mirdtr]].values
feature_mirdtr

In [ ]:
mirdtr_df = pd.concat([df[feature_mirdtr], df['labels']], axis=1)

# 加载数据集并进行标准化处理
X_mirdtr = mirdtr_df.drop(['labels'], axis = 1).values
y_mirdtr = mirdtr_df['labels'].values
scaler = StandardScaler()
X_scaled_mirdtr = scaler.fit_transform(X_mirdtr)

X_resampled, y_resampled = resample(X_scaled_mirdtr, y_mirdtr, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
pred_dt = automl.predict(X_test_resampled)
accuracy = accuracy_score(y_test_resampled, pred_dt).round(5)
precision = precision_score(y_test_resampled, pred_dt).round(5)
recall = recall_score(y_test_resampled, pred_dt).round(5) 
f1 = f1_score(y_test_resampled, pred_dt).round(5)
roc_auc = roc_auc_score(y_test_resampled, pred_dt).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')

In [ ]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge
from xgboost import XGBClassifier as XGB
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import cross_val_score


class ImprovedREFCV:
    def __init__(self, X, y, n_features, step=1, cv=5, alpha_lasso=0.01, alpha_ridge=0.01):
        """
        初始化函数，输入参数：
        X: 数据集特征矩阵（ndarray）
        y: 标签向量（ndarray）
        n_features: 选择的特征数量（int）
        step: 每轮迭代中选择特征的步长（int，默认为1）
        cv: 交叉验证折数（int，默认为5）
        alpha_lasso: L1正则化系数（float，默认为0.01）
        alpha_ridge: L2正则化系数（float，默认为0.01）
        """
        self.X = X
        self.y = y
        self.n_features_ = n_features
        self.step_ = step
        self.cv_ = cv
        self.alpha_lasso = alpha_lasso
        self.alpha_ridge = alpha_ridge
        self.selected_features_ = []

    def _calculate_stability(self, model, X, y, n_iter=10):
        """
        计算稳定性得分的内部函数。该函数使用稳定性评估方法对给定的特征子集进行评估，并返回一个稳定性得分。
        输入参数：
        model: 回归模型实例
        X: 特征矩阵（ndarray）
        y: 标签向量（ndarray）
        n_iter: 采样次数（int，默认为10）
        """
        scores = []
        for i in range(n_iter):
            # 采样训练和测试数据
            idx_train = np.random.choice(X.shape[0], size=int(0.7 * X.shape[0]), replace=False)
            idx_test = np.setdiff1d(np.arange(X.shape[0]), idx_train)
            X_train, y_train = X[idx_train, :], y[idx_train]
            X_test, y_test = X[idx_test, :], y[idx_test]

            # 使用回归模型进行拟合和预测，并计算R方值
            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)

            scores.append(score)

        return np.mean(scores)

    def fit_transform(self):
        """
        特征选择函数，返回被选择的特征的索引列表。
        """
        # 构建特征集合
        features = np.arange(self.X.shape[1])

        for i in range(self.n_features_):
            # 初始化累计评分和稳定性得分
            scores = np.zeros(features.shape[0])
            stabilities = np.zeros(features.shape[0])
            lasso_regularizations = np.zeros(features.shape[0])
            ridge_regularizations = np.zeros(features.shape[0])

            # 计算每个特征的评分和稳定性得分
            for j, feature in enumerate(features):
                # 对于每个回归模型，使用交叉验证计算特征子集的平均R方值
                models = [
                    RandomForestClassifier(random_state=42),
                    XGB(random_state=42)
                ]
                scores_model = []
                stabilities_model = []
                lasso_regularization_model = []
                ridge_regularization_model = []
                
                for model in models:
                    scores_cv = cross_val_score(model, self.X[:, feature][:, np.newaxis], self.y, cv=self.cv_,
                                                scoring='accuracy')
                    
                    score = np.mean(scores_cv)
                    stability = self._calculate_stability(model, self.X[:, feature][:, np.newaxis], self.y)

                    # 计算L1正则化项得分
                    if self.alpha_lasso > 0:
                        model_lasso = Lasso(alpha=self.alpha_lasso)
                        model_lasso.fit(self.X[:,feature][:,np.newaxis],self.y)
                        regularization_score_lasso = np.abs(model_lasso.coef_[0])
                    else:
                        regularization_score_lasso = 0

                    # 计算L2正则化项得分
                    if self.alpha_ridge > 0:
                        model_ridge = Ridge(alpha=self.alpha_ridge)
                        model_ridge.fit(self.X[:,feature][:,np.newaxis],self.y)
                        regularization_score_ridge = np.abs(model_ridge.coef_[0])
                    else:
                        regularization_score_ridge = 0

                    scores_model.append(score)
                    stabilities_model.append(stability)
                    lasso_regularization_model.append(regularization_score_lasso)
                    ridge_regularization_model.append(regularization_score_ridge)

                # 将两个回归模型的评分取平均值作为最终评分
                scores[j] = np.mean(scores_model)
                # print("第", j+1, "个特征的得分：", scores[j])
                stabilities[j] = np.mean(stabilities_model)
                # print("第", j+1, "个特征的稳定性得分：", stabilities[j])
                lasso_regularizations[j] = np.mean(lasso_regularization_model)
                # print("第", j+1, "个特征的L1得分：", lasso_regularizations[j])
                ridge_regularizations[j] = np.mean(ridge_regularization_model)
                # print("第", j+1, "个特征的L2得分：", ridge_regularizations[j])

            # 计算得分最高的特征索引
            best_score_idx = np.argmax(scores + stabilities + lasso_regularizations + ridge_regularizations)
            best_feature = features[best_score_idx]
            self.selected_features_.append(best_feature)
            # print('第', i+1, '个特征：', best_feature) #输出每次选中的特征

            # 更新特征集合，移除选出的特征和相邻的step-1个特征
            idx_remove = np.arange(max(0, best_score_idx - self.step_ + 1), min(best_score_idx + self.step_, features.shape[0]))
            # print("移除特征：", idx_remove)
            features = np.setdiff1d(features, features[idx_remove])
            # print('剩余的特征：', features) #输出剔除后的特征

        return self.selected_features_

In [ ]:
refcv = ImprovedREFCV(X_scaled, y, n_features=12, alpha_lasso=0.01, alpha_ridge=0.01)

# 执行特征选择
selected_features = refcv.fit_transform()

# 输出结果
print('Selected features: ', selected_features)

In [ ]:
feature1 = df.columns.drop(['labels'])[[selected_features]].values
feature1

In [ ]:
improvedRFECV = pd.concat([df[feature1], df['labels']], axis=1)
X_improvedRFECV = improvedRFECV.drop(['labels'], axis = 1).values
y_improvedRFECV = improvedRFECV['labels'].values
scaler = StandardScaler()
X_scaled_improvedRFECV = scaler.fit_transform(X_improvedRFECV)

X_resampled, y_resampled = resample(X_scaled_improvedRFECV, y_improvedRFECV, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
pred_imrfecv = automl.predict(X_test_resampled)
accuracy = accuracy_score(y_test_resampled, pred_imrfecv).round(5)
precision = precision_score(y_test_resampled, pred_imrfecv).round(5)
recall = recall_score(y_test_resampled, pred_imrfecv).round(5) 
f1 = f1_score(y_test_resampled, pred_imrfecv).round(5)
roc_auc = roc_auc_score(y_test_resampled, pred_imrfecv).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import lightgbm as lgbm

lgb = lgbm.LGBMClassifier(random_state=42)

# 构造SFS算法对象，初始化特征子集为空集
sfs = SFS(lgb, k_features='best', forward=True, scoring='accuracy', cv=5)

# 将数据X和标签y作为输入，进行特征选择
sfs.fit(X_scaled, y)

# 输出最佳特征子集
print('Best feature subset:', sfs.k_feature_idx_)


In [ ]:
best_features = sfs.k_feature_idx_
sfsf_feature = df.columns.drop(['labels'])[[best_features]].values
sfsf_feature

In [ ]:
sfsf = pd.concat([df[sfsf_feature], df['labels']], axis=1)
X_sfsf = sfsf.drop(['labels'], axis = 1).values
y_sfsf = sfsf['labels'].values
scaler = StandardScaler()
X_scaled_sfsf = scaler.fit_transform(X_sfsf)

X_resampled, y_resampled = resample(X_scaled_sfsf, y_sfsf, replace=True, n_samples=len(y==1), random_state=42)  
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [ ]:
automl = AutoSklearnClassifier(
    time_left_for_this_task=120*5,
    per_run_time_limit=30,
    metric=autosklearn.metrics.accuracy,
    seed=42,
    resampling_strategy='cv',
    resampling_strategy_arguments={'folds': 5}
)
automl.fit(X_train_resampled, y_train_resampled)
automl.leaderboard(detailed = True, ensemble_only=False)
print(automl.sprint_statistics())

In [ ]:
print(automl.show_models())

In [ ]:
pred_sfsf = automl.predict(X_test_resampled)
accuracy = accuracy_score(y_test_resampled, pred_sfsf).round(5)
precision = precision_score(y_test_resampled, pred_sfsf).round(5)
recall = recall_score(y_test_resampled, pred_sfsf).round(5) 
f1 = f1_score(y_test_resampled, pred_sfsf).round(5)
roc_auc = roc_auc_score(y_test_resampled, pred_sfsf).round(5)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')
print(f'ROC_AUC: {roc_auc}')